In [ ]:
#imports
from typing import List, Dict, Optional
import openai
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import textwrap as tr
import time
import ast
from IPython.display import display, Image, HTML
from datetime import datetime, timedelta

pd.set_option("display.max_colwidth", None)

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/llms_final_project')
#os.listdir()

['IMDB Dataset.csv',
 'filtered_reviews.csv',
 'interim_results_gpt-4o-mini_0.csv',
 'interim_results_gpt-3.5-turbo-1106_0.csv',
 'final_results_claude-3-5-sonnet-20241022_0_to_1001.csv',
 'claude_missing_reviews.csv',
 'claude_complete_results.csv',
 'final_results_unholy_0_to_5.csv',
 '=4.32.0',
 'final_results_nsfw_0_to_1.csv',
 'final_results_nsfw_1_to_4.csv',
 'final_results_nsfw_5_to_6.csv',
 'interim_results_nsfw_0_to_99.csv',
 'interim_results_nsfw_0_to_199.csv',
 'interim_results_nsfw_0_to_299.csv',
 'interim_results_nsfw_0_to_399.csv',
 'interim_results_nsfw_0_to_499.csv',
 'final_results_nsfw_0_to_5.csv',
 'final_results_nsfw_0_to_10.csv',
 'IMDB Dataset.gsheet',
 'final_results_claude-3-5-sonnet-20241022_0_to_1001.gsheet',
 'master1000.csv']

In [ ]:
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [ ]:
from google.colab import userdata
import os
my_openaikey = userdata.get('tauhid')   #Personal OpenAI API Key
#my_cohere_key = userdata.get('MAKE THIS RED TEXT MATCH YOUR COHERE SECRET KEY')   #Key for Cohere Embeddings

#With these personal keys, you can call the llm model without keys
os.environ['OPENAI_API_KEY'] = my_openaikey
#os.environ['COHERE_API_KEY'] = my_cohere_key

In [ ]:
def get_completion(prompt, instructions, client, model="gpt-4o",
                   output_type = 'text'):
  '''Get a text completion from the OpenAI API'''
  completion = client.chat.completions.create(
                model=model,
                response_format={ "type": output_type},
                messages=[
                  {"role": "system", "content": instructions},
                  {"role": "user", "content": prompt}
                ]
              )
  response =completion.choices[0].message.content

  return response


In [ ]:
df['review'].iloc[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
instructions = '''You will receive a text excerpt. Please write a new passage of similar length that covers the same key points and maintains a similar style, but using your own words:
Write a new version that:
1. Maintains similar length
2. Covers the same main ideas
3. Uses similar tone/style
4. Feels natural and coherent
Return only your new version'''

In [ ]:
test = get_completion(df['review'].iloc[0], instructions, client, model = 'gpt-3.5-turbo-1106')

In [ ]:
print(test)

After just one episode of Oz, I was immediately drawn in, as the intensity and unapologetic portrayal of violence captivated me from the start. This is not a show for those with a faint heart. It delves into the harsh realities of drugs, sex, and brutality without holding back.

Set in the Oswald Maximum Security State Penitentiary, or OZ, the series primarily focuses on Emerald City, an experimental unit where privacy is minimal and tensions run high among a diverse array of inmates – Aryans, Muslims, gangstas, Latinos, Christians, Italians, and more. The constant threat of conflict and illicit activities creates a volatile environment.

What sets Oz apart is its willingness to confront the darkest aspects of human nature without pretense or sugar-coating. There's no room for pretty illusions or romanticized portrayals here. While the explicit violence initially caught me off guard, I soon found myself acclimating to its unflinching depiction. The show doesn’t shy away from the harsh 

#Text Preprocessing

In [ ]:
df["length"]=df['review'].apply(lambda x: len(x))

In [ ]:
df["length"].describe()

,length
count,50000.000000
mean,1309.431020
std,989.728014
min,32.000000
25%,699.000000
50%,970.000000
75%,1590.250000
max,13704.000000


We have a right-skewed distribution with high variance. I'm going to filter this down to 10,000 random reviews within a "reasonable range" around the median. Let's go 200 characters in each direction, so 770 to 1170.

In [ ]:
# Define reasonable length bounds
min_chars = 770
max_chars = 1170

# Filter dataset
filtered_df = df[
    (df['length'] >= min_chars) &
    (df['length'] <= max_chars)
].copy()

# Randomly sample 10,000 reviews
final_sample = filtered_df.sample(n=10000, random_state=42)

In [ ]:
final_sample.head()

,review,sentiment,length
30237,"Lately they have been trying to hock this film late-night on cable TV commercials. Don't believe the hype. I was one of the unlucky people to see this stinker in theatres. This is, in my opinion, the 3rd Worst Movie of All Time, just behind Mac & Me (#1 Worst), and Jack Frost (#2 Worst), but I must admit, they are all close and all TERRIBLE! Really, nothing of this movie is funny, or disturbing, or anything else it claims to be so don't waste your money. The only thing it is good for is giving to your worst enemy. I'm not lying about that. Someone who you would love to kill or torture would be a prime candidate for this film. It is that awful. If you don't believe me then you deserve to suffer through the misery of watching this, which I doubt you can finish. Two Thumbs Enthusiastically Down.",negative,803
14977,"If the scale went negative I would be happier. Seeing Sushmita Sen was nice, and Nisha Kothari has a bright future but the producer and the director ruined any and all enjoyment in this story. The choice of angles, choice of lighting and well everything distracted from trying to remember what is the story. Oh, if the songs and dances haven't caused you to rip your ears off your head, first. The film could have been made twice at 1.25 hours, and been pretty good, kinda like ""Seven Samurai"" but the director and writer didn't go that direction, even if the ""townsfolk"" finally find their backbone and want to help. This movie fails on so many levels: editing, writing, photography angles, style, lighting, script - name any aspect of this film - it was BAD - probably the food from the caterer was bad too. I have never in 6 years of watching Indian (Bollywood) movies seen something this badly made.",negative,903
7363,"Rachael Ray appeals to viewers of all ages and backgrounds, beginner cooks or ""seasoned"" veterans. You'll be dazzled with a variegated presentation of delectable yet time-efficient dishes, jazzed up with her unique brand of spunk and candor. Most importantly, this hip chic keeps her audience drawn in by stimulating all five senses. Let me explain. Her program provides enlightenment to your visual sense, auditory sense, and sense of feeling through a rich, luminous ambient backdrop, light-hearted, casual, yet engaging topics, eye-pleasing, appetite wrenching meals, and her hearty smile and laugh, which will simmer down anyone's nerves.(Sense of smell and taste are rewarded when you test out the recipes in your own kitchen and among your own family and friends). Check out her show guys.",positive,795
43930,"Someone had a great idea: let's have Misty Mundae do her own, R-rated version of Lara Croft - firing two guns not only in skimpy outfits, but topless as well. It WAS indeed a great idea. The problem is that the people who had it couldn't come up with any sort of script or budget to support it. Therefore, we get a ""film"" that barely reaches medium length by replaying many of its parts (often in slow-motion), and was apparently shot entirely inside a garage. The appeal of Misty Mundae is still evident: she is unbelievably cute and has a natural girl-next-door beauty. However her two female co-stars here, with whom she shares a lengthy lesbian scene, are nowhere near her league. If ""Mummy Raider"" was presented as a Youtube video, I'd rate it higher, but as a ""film"" destined for DVD consumption it cannot get more than 1/2 a * out of 4.",negative,843
3262,"This film is the worst excuse for a motion picture I have EVER seen. To begin, I'd like to say the the front cover of this film is by all means misleading, if you think you are about to see a truly scary horror film with a monster clown, you are soooo wrong. In fact the killers face doesn't even slightly resemble the front cover, it's just an image they must have found on Google and thought it looked cool. Speaking of things they found and thought it looked cool, there is a scene in this film where some of the gang are s

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('filtered_reviews.csv', index=True)

In [ ]:
import re

In [ ]:
def clean_text(text):
    # Replace single or multiple <br /> tags with a single newline
    # This regex will catch both <br /> and any number of them in sequence
    cleaned = re.sub(r'(<br />)+', '\n', text)

    # Remove any extra whitespace that might have been created
    cleaned = ' '.join(cleaned.split())

    return cleaned

# Apply to the dataset
filtered_df['cleaned_review'] = filtered_df['review'].apply(clean_text)

# Quick check
print("Original:")
print(filtered_df['review'].iloc[0])
print("\nCleaned:")
print(filtered_df['cleaned_review'].iloc[0])

Original:
A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly we

In [ ]:
print("Original:")
print(filtered_df['review'].iloc[180])
print("\nCleaned:")
print(filtered_df['cleaned_review'].iloc[180])

Original:
I saw it at the Legacy Theater in the Joseph Smith Memorial Building in Salt Lake City this morning. I'm going to assume that one's level of enjoyment during this movie will largely be based on one's level of acceptance of Joseph's story.<br /><br />However, that aside it was very well made, well acted, and had a nice score. If you get to Salt Lake City, it is a must to see it in the Legacy Theater. I have never been in a nicer theater as far as picture quality, sound quality and ambiance in my entire life...I wonder if the Church would let me watch Batman Begins there! Being that I'm LDS and regard Joseph as a prophet, I was touched in several places and was brought to tears quite a few times...which I presume is expected since they handed out tissues BEFORE the movie started! Anyway, I'm told that this film is available in several LDS Visitor Centers around the globe, if you have 70 minutes check it out because whether you believe Joseph Smith or not, he tells a fascinating

In [ ]:
filtered_df = filtered_df.sample(n=10000, random_state=42)

In [ ]:
# Save the DataFrame to a CSV file
filtered_df.to_csv('filtered_reviews.csv', index=True)

In [ ]:
filtered_df = pd.read_csv("filtered_reviews.csv")
df = pd.read_csv("IMDB Dataset.csv")

100%|██████████| 1001/1001 [26:59<00:00,  1.62s/it]


In [ ]:
filtered_df = pd.read_csv("filtered_reviews.csv")
df = pd.read_csv("IMDB Dataset.csv")


In [ ]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 11.2 MB/s eta 0:00:00


In [ ]:
import anthropic
client = anthropic.Anthropic(api_key="redacted for github")

In [ ]:
def process_reviews_claude(df, n_samples, model_name, instructions, start_idx=0):
    """
    Process a batch of reviews through Claude API.

    Args:
        df: DataFrame containing reviews
        n_samples: Number of reviews to process
        model_name: Name of the Claude model to use
        instructions: Instructions for Claude
        start_idx: Starting index in the DataFrame
    """
    results = []

    for idx in tqdm(range(start_idx, start_idx + n_samples)):
        try:
            original = df['cleaned_review'].iloc[idx]

            # Combine instructions and prompt for Claude
            combined_prompt = f"{instructions}\n\nText excerpt:\n{original}"

            # Get Claude generation
            completion = client.messages.create(
                model=model_name,
                max_tokens=1024,
                messages=[
                    {"role": "user", "content": combined_prompt}
                ]
            )

            # Extract the text content from Claude's response
            generated = completion.content[0].text  # Change is here

            # Store result
            result = {
                'original_idx': df.index[idx],
                'original_review': original,
                f'generated_{model_name}': generated,
                'original_length': len(original.split()),
                'generated_length': len(generated.split()),
                'timestamp': datetime.now().isoformat()
            }

            results.append(result)

            # Save intermediate results every 1000 samples
            if len(results) % 1000 == 0:
                interim_df = pd.DataFrame(results)
                interim_df.to_csv(f'interim_results_{model_name}_{start_idx}_to_{idx}.csv', index=False)

            # Add a small delay to avoid rate limits
            time.sleep(0.1)

        except Exception as e:
            print(f"Error processing review {idx}: {str(e)}")
            continue

    # Save final results
    final_df = pd.DataFrame(results)
    final_df.to_csv(f'final_results_{model_name}_{start_idx}_to_{start_idx+n_samples}.csv', index=False)

    return final_df

# Let's test with a smaller sample first
instructions = '''Rewrite the following text excerpt in your own words. Your response should:
1. Maintain similar length
2. Cover the same main ideas
3. Use similar tone/style
4. Feel natural and coherent

Important: Return ONLY the rewritten text with no additional commentary, introductions, or explanations.'''
n_samples = 1001  # Start with 5 to test
results_df = process_reviews_claude(filtered_df, n_samples, 'claude-3-5-sonnet-20241022', instructions)

  3%|▎         | 28/1001 [02:04<1:05:28,  4.04s/it]

Error processing review 27: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}


 22%|██▏       | 223/1001 [16:48<1:00:26,  4.66s/it]

Error processing review 222: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}


 32%|███▏      | 321/1001 [24:33<54:54,  4.85s/it]

...

 77%|███████▋  | 769/1001 [1:00:45<18:08,  4.69s/it]

Error processing review 768: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}


 78%|███████▊  | 779/1001 [1:01:30<17:17,  4.68s/it]

Error processing review 778: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}


100%|██████████| 1001/1001 [1:17:56<00:00,  4.67s/it]


In [ ]:
results_df

,original_idx,original_review,generated_claude-3-5-sonnet-20241022,original_length,generated_length,timestamp
0,0,"Lately they have been trying to hock this film late-night on cable TV commercials. Don't believe the hype. I was one of the unlucky people to see this stinker in theatres. This is, in my opinion, the 3rd Worst Movie of All Time, just behind Mac & Me (#1 Worst), and Jack Frost (#2 Worst), but I must admit, they are all close and all TERRIBLE! Really, nothing of this movie is funny, or disturbing, or anything else it claims to be so don't waste your money. The only thing it is good for is giving to your worst enemy. I'm not lying about that. Someone who you would love to kill or torture would be a prime candidate for this film. It is that awful. If you don't believe me then you deserve to suffer through the misery of watching this, which I doubt you can finish. Two Thumbs Enthusiastically Down.","They've started pushing this movie on late-night TV with those desperate infomercials, but don't fall for it. I had the misfortune of catching this disaster in an actual theatre, and it ranks as my number three worst film ever, right after Mac & Me and Jack Frost - though honestly, they're all neck-and-neck in their awfulness. The movie fails completely at everything it attempts - it's not funny, not scary, not shocking, nothing. Save your money. The only conceivable use for this film would be as a form of punishment for someone you truly despise. I mean that sincerely - if you have a mortal enemy, make them watch this. If you're skeptical of my warning and decide to watch it anyway, well, you'll deserve the torment that awaits. I'd be shocked if you even make it to the end. A resounding, emphatic thumbs down.",152,145,2024-12-01T06:18:29.313679
1,1,"If the scale went negative I would be happier. Seeing Sushmita Sen was nice, and Nisha Kothari has a bright future but the producer and the director ruined any and all enjoyment in this story. The choice of angles, choice of lighting and well everything distracted from trying to remember what is the story. Oh, if the songs and dances haven't caused you to rip your ears off your head, first. The film could have been made twice at 1.25 hours, and been pretty good, kinda like ""Seven Samurai"" but the director and writer didn't go that direction, even if the ""townsfolk"" finally find their backbone and want to help. This movie fails on so many levels: editing, writing, photography angles, style, lighting, script - name any aspect of this film - it was BAD - probably the food from the caterer was bad too. I have never in 6 years of watching Indian (Bollywood) movies seen something this badly made.","While it was nice to see Sushmita Sen, and newcomer Nisha Kothari shows promise, this film is so terrible it deserves a rating below zero. The production team completely destroyed any potential entertainment value. Everything from the camera work to the lighting choices serves as a constant distraction from the plot - that is, if you haven't already fled the theater due to the unbearable song and dance numbers. There could have been a decent movie here, perhaps following a ""Seven Samurai"" style narrative (especially given how the villagers eventually find their courage), but they missed that opportunity entirely. Instead, we got a 2.5-hour disaster that fails in literally every department: the editing is atrocious, the cinematography is amateur, the writing is poor, and the overall execution is dreadful. I wouldn't be surprised if even the catering was subpar. In my six years following Bollywood cinema, I've never encountered a film this incompetently made.",161,154,2024-12-01T06:18:33.479204
2,2,"Rachael Ray appeals to viewers of all ages and backgrounds, beginner cooks or ""seasoned"" veterans. You'll be dazzled with a variegated presentation of delectable yet time-efficient dishes, jazzed up with her unique brand of spunk and candor. Most importantly, this hip chic keeps her audie

In [ ]:
def process_missing_reviews_claude(df, failed_indices, model_name, instructions):
    """
    Process only the missing reviews from failed attempts.

    Args:
        df: DataFrame containing reviews
        failed_indices: List of indices that failed in the first run
        model_name: Name of the Claude model to use
        instructions: Instructions for Claude
    """
    results = []

    for idx in tqdm(failed_indices):
        try:
            original = df['cleaned_review'].iloc[idx]

            # Combine instructions and prompt for Claude
            combined_prompt = f"{instructions}\n\nText excerpt:\n{original}"

            # Get Claude generation
            completion = client.messages.create(
                model=model_name,
                max_tokens=1024,
                messages=[
                    {"role": "user", "content": combined_prompt}
                ]
            )

            # Extract the text content from Claude's response
            generated = completion.content[0].text

            # Store result
            result = {
                'original_idx': df.index[idx],
                'original_review': original,
                f'generated_{model_name}': generated,
                'original_length': len(original.split()),
                'generated_length': len(generated.split()),
                'timestamp': datetime.now().isoformat()
            }

            results.append(result)

            # Longer delay to avoid overload
            time.sleep(1.0)  # Increased delay

        except Exception as e:
            print(f"Error processing review {idx}: {str(e)}")
            continue

    return pd.DataFrame(results)

# List of failed indices
failed_indices = [27, 222, 320, 371, 414, 447, 498, 511, 550, 580, 590,
                 629, 640, 654, 672, 684, 768, 778]

# Process just these indices
missing_results_df = process_missing_reviews_claude(
    filtered_df,
    failed_indices,
    'claude-3-5-sonnet-20241022',
    instructions
)

# Save these results separately
missing_results_df.to_csv('claude_missing_reviews.csv', index=False)

100%|██████████| 18/18 [01:43<00:00,  5.74s/it]


In [ ]:
# Load the main results and missing results
main_results = pd.read_csv('final_results_claude-3-5-sonnet-20241022_0_to_1001.csv')
missing_results = pd.read_csv('claude_missing_reviews.csv')

# Merge on original_idx
merged_df = pd.concat([main_results, missing_results]).sort_values('original_idx')

# Remove any duplicates if they exist, keeping the latest version
merged_df = merged_df.drop_duplicates(subset='original_idx', keep='last')

# Sort by original_idx to maintain order
merged_df = merged_df.sort_values('original_idx').reset_index(drop=True)

# Verify we have all 1001 rows
print(f"Total rows after merge: {len(merged_df)}")

# Save the complete dataset
merged_df.to_csv('claude_complete_results.csv', index=False)

Total rows after merge: 1001
